In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from simility_apis.return_rule_info_from_simility import ReturnRuleInfoFromSimilityAPI
from simility_requests.return_rule_info_from_simility import ReturnRuleInfoFromSimility
# from simility_requests.return_cassandra_dtypes import ReturnCassandraDatatypes
from simility_apis.set_password import set_password
import pandas as pd
import numpy as np
import json

In [5]:
set_password()
params = {
    'url': 'https://app.simility.com',
    'app_prefix': 'republic_wireless',
    'user': 'james@simility.com',
    'entity': 'transaction'
}
# rri = ReturnRuleInfoFromSimilityAPI(**params)
rri = ReturnRuleInfoFromSimility(**params)
rule_info = rri.request()

Please provide your password for logging into the Simility platform:  ········


In [6]:
# test_sys_rules = json.loads(rule_info)['entries']

In [7]:
rule_conditions = rule_info['conditions'].to_dict()

In [13]:
# rule_conditions

In [16]:
# {
#     'createdBy': None,
#     'createdOn': None,
#     'appPrefix': self.app_prefix,
#     'entityName': self.entity,
#     'name': None,
#     'conditions': None,
#     'score': None,
#     'status': self.status,
#     'isRun': False,
#     'ruleGroupNames': None,
#     'isAutoGenerated': True
# }

In [18]:
# from datetime import datetime
# now = datetime.now()
# now.strftime("%Y-%m-%dT%H:%M:%S")

In [20]:
import random

In [21]:
dummy_scores = {}
for rule_name, _ in rule_conditions.items():
    dummy_scores[rule_name] = random.randint(-100, 100)

In [30]:
from generate_system_configs.create_new_configs import CreateNewConfigs

In [31]:
c = CreateNewConfigs(conditions=rule_conditions, scores=dummy_scores, app_prefix='republic_wireless', entity='transaction', make_active=True)

In [32]:
c.generate()

{'IP in VE': {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-15T15:17:06',
  'appPrefix': 'republic_wireless',
  'entityName': 'transaction',
  'name': 'IP in VE',
  'conditions': '"{\\"condition\\":\\"OR\\",\\"rules\\":[{\\"id\\":\\"ip_country_code\\",\\"field\\":\\"ip_country_code\\",\\"type\\":\\"string\\",\\"operator\\":\\"equal\\",\\"value\\":\\"value.VE\\",\\"data\\":{}},{\\"id\\":\\"ip_country_code\\",\\"field\\":\\"ip_country_code\\",\\"type\\":\\"string\\",\\"operator\\":\\"equal\\",\\"value\\":\\"value.AU\\",\\"data\\":{}},{\\"id\\":\\"ip_country_code\\",\\"field\\":\\"ip_country_code\\",\\"type\\":\\"string\\",\\"operator\\":\\"equal\\",\\"value\\":\\"value.CO\\",\\"data\\":{}},{\\"id\\":\\"ip_country_code\\",\\"field\\":\\"ip_country_code\\",\\"type\\":\\"string\\",\\"operator\\":\\"equal\\",\\"value\\":\\"value.DO\\",\\"data\\":{}},{\\"id\\":\\"ip_country_code\\",\\"field\\":\\"ip_country_code\\",\\"type\\":\\"string\\",\\"operator\\":\\"equal\\",\\"value\\":\\"

# Test on dummy data from sim-ds env

In [19]:
from simility_apis.set_password import set_password
set_password()

Please provide your password for logging into the Simility platform:  ·········


In [20]:
params = {
    'url': 'http://sim-ds.us-central1.gcp.dev.paypalinc.com/', 
    'app_prefix': 'james_testing', 
    'user': 'james@simility.com',
    'entity': 'transaction'
}

In [21]:
# X = pd.read_csv('dummy_pipeline_output_data.csv', index_col='eid')

In [22]:
rules_strings = {
    'Rule1': "(X['account_number_avg_order_total_per_account_number_1day']>1)",
    'Rule2': "(X['sim_sc_ml']>0.5)",
    'Rule3': "(X['is_billing_shipping_city_same']==False)",
}

In [23]:
field_datatypes = {
    'account_number_avg_order_total_per_account_number_1day': 'DOUBLE',
    'sim_sc_ml': 'FLOAT',
    'is_billing_shipping_city_same': 'BOOLEAN',
}
cassandra_field_names = {
    'account_number_avg_order_total_per_account_number_1day': 'account_number.avg_order_total_per_account_number_1day',
    'sim_sc_ml': 'sim_sc_ml',
    'is_billing_shipping_city_same': 'is_billing_shipping_city_same',
}

In [24]:
from rules.rules import Rules

In [25]:
r = Rules(rule_strings=rules_strings)

In [26]:
r.as_system_dicts(field_datatypes, cassandra_field_names)

{'Rule1': {'condition': 'AND',
  'rules': [{'field': 'account_number.avg_order_total_per_account_number_1day',
    'operator': 'greater',
    'value': 'value.1.0',
    'id': 'account_number.avg_order_total_per_account_number_1day',
    'type': 'double'}]},
 'Rule2': {'condition': 'AND',
  'rules': [{'field': 'sim_sc_ml',
    'operator': 'greater',
    'value': 'value.0.5',
    'id': 'sim_sc_ml',
    'type': 'float'}]},
 'Rule3': {'condition': 'AND',
  'rules': [{'field': 'is_billing_shipping_city_same',
    'operator': 'equal',
    'value': '0',
    'id': 'is_billing_shipping_city_same',
    'type': 'boolean',
    'input': 'radio'}]}}

In [27]:
dummy_scores = {
    'Rule1': -10,
    'Rule2': -5,
    'Rule3': -2,
}

In [28]:
c = CreateNewConfigs(conditions=r.system_dicts, 
                     scores=dummy_scores, 
                     app_prefix=params['app_prefix'], 
                     entity=params['entity'], 
                     make_active=True)

In [29]:
rule_configs = c.generate()

In [30]:
rule_configs

{'Rule1': {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T15:25:26',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule1',
  'conditions': '{"condition": "AND", "rules": [{"field": "account_number.avg_order_total_per_account_number_1day", "operator": "greater", "value": "value.1.0", "id": "account_number.avg_order_total_per_account_number_1day", "type": "double"}]}',
  'score': -10,
  'status': 'ACTIVE',
  'isAutoGenerated': True},
 'Rule2': {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T15:25:26',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule2',
  'conditions': '{"condition": "AND", "rules": [{"field": "sim_sc_ml", "operator": "greater", "value": "value.0.5", "id": "sim_sc_ml", "type": "float"}]}',
  'score': -5,
  'status': 'ACTIVE',
  'isAutoGenerated': True},
 'Rule3': {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T15:25:26',
  'appPrefix': 'james_testing',
  'entityName': 'tran

In [31]:
# rule_configs_str = {}

In [32]:
# for rule_name, rule_config in rule_configs.items():
#     rule_configs_str[rule_name] = json.dumps(rule_config)

In [33]:
# rule_configs_str

In [56]:
from simility_requests.create_rules_in_simility import CreateRulesInSimility

In [57]:
cr = CreateRulesInSimility(url=params['url'], 
                           app_prefix=params['app_prefix'], 
                           user=params['user'])

In [58]:
cr.request(rule_configs=rule_configs)

In [45]:
list(rule_configs.values())

[{'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T15:25:26',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule1',
  'conditions': '{"condition": "AND", "rules": [{"field": "account_number.avg_order_total_per_account_number_1day", "operator": "greater", "value": "value.1.0", "id": "account_number.avg_order_total_per_account_number_1day", "type": "double"}]}',
  'score': -10,
  'status': 'ACTIVE',
  'isAutoGenerated': True},
 {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T15:25:26',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule2',
  'conditions': '{"condition": "AND", "rules": [{"field": "sim_sc_ml", "operator": "greater", "value": "value.0.5", "id": "sim_sc_ml", "type": "float"}]}',
  'score': -5,
  'status': 'ACTIVE',
  'isAutoGenerated': True},
 {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T15:25:26',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule3'

# unit testing

In [14]:
from system_config_generation.create_new_configs import CreateNewConfigs

In [15]:
system_dicts = {'Rule1': {'condition': 'AND',
                              'rules': [{'condition': 'OR',
                                         'rules': [{'id': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
                                                    'field': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
                                                    'type': 'double',
                                                    'operator': 'greater_or_equal',
                                                    'value': 'value.60.0'},
                                                   {'id': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
                                                    'field': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
                                                    'type': 'double',
                                                    'operator': 'greater',
                                                    'value': 'value.120.0'},
                                                   {'id': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
                                                    'field': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
                                                    'type': 'double',
                                                    'operator': 'less_or_equal',
                                                    'value': 'value.500.0'}],
                                         },
                                        {'id': 'num_items',
                                         'field': 'num_items',
                                         'type': 'integer',
                                         'operator': 'equal',
                                         'value': 'value.1'},
                                        ]},
                    'Rule2': {'condition': 'AND',
                              'rules': [{'id': 'ml_cc_v0',
                                         'field': 'ml_cc_v0',
                                         'type': 'double',
                                         'operator': 'less',
                                         'value': 'value.0.315'},
                                        {'condition': 'OR',
                                         'rules': [{'id': 'method_clean',
                                                    'field': 'method_clean',
                                                    'type': 'string',
                                                    'operator': 'equal',
                                                    'value': 'value.checkout'},
                                                   {'id': 'method_clean',
                                                    'field': 'method_clean',
                                                    'type': 'string',
                                                    'operator': 'begins_with',
                                                    'value': 'value.checkout'},
                                                   {'id': 'method_clean',
                                                    'field': 'method_clean',
                                                    'type': 'string',
                                                    'operator': 'ends_with',
                                                    'value': 'value.checkout'},
                                                   {'id': 'method_clean',
                                                    'field': 'method_clean',
                                                    'type': 'string',
                                                    'operator': 'contains',
                                                    'value': 'value.checkout'},
                                                   {'id': 'ip_address',
                                                    'field': 'ip_address',
                                                    'type': 'string',
                                                    'operator': 'is_not_null',
                                                    'value': None},
                                                   {'id': 'ip_isp',
                                                    'field': 'ip_isp',
                                                    'type': 'string',
                                                    'operator': 'is_not_empty',
                                                    'value': None}
                                                   ]}]},
                    'Rule3': {'condition': 'AND',
                              'rules': [{'id': 'method_clean',
                                         'field': 'method_clean',
                                         'type': 'string',
                                         'operator': 'not_begins_with',
                                         'value': 'value.checkout'},
                                        {'id': 'method_clean',
                                         'field': 'method_clean',
                                         'type': 'string',
                                         'operator': 'not_ends_with',
                                         'value': 'value.checkout'},
                                        {'id': 'method_clean',
                                         'field': 'method_clean',
                                         'type': 'string',
                                         'operator': 'not_contains',
                                         'value': 'value.checkout'},
                                        {'condition': 'OR',
                                         'rules': [{'id': 'ip_address',
                                                    'field': 'ip_address',
                                                    'type': 'string',
                                                    'operator': 'is_null',
                                                    'value': None},
                                                   {'id': 'ip_isp',
                                                    'field': 'ip_isp',
                                                    'type': 'string',
                                                    'operator': 'is_empty',
                                                    'value': None},
                                                   ],
                                         }]},
                    'Rule4': {'condition': 'AND',
                              'rules': [{'id': 'forwarder_address',
                                         'field': 'forwarder_address',
                                         'type': 'boolean',
                                         'input': 'radio',
                                         'operator': 'equal',
                                         'value': '1'},
                                        {'id': 'is_shipping_billing_address_same',
                                         'field': 'is_shipping_billing_address_same',
                                         'type': 'boolean',
                                         'input': 'radio',
                                         'operator': 'equal',
                                         'value': '0'}]},
                    'Rule5': {'condition': 'AND',
                              'rules': [{'id': 'ad_price_type',
                                         'field': 'ad_price_type',
                                         'type': 'string',
                                         'operator': 'not_in',
                                         'value': 'value.FREE,NEGOTIATION'},
                                        {'id': 'ad_price_type',
                                         'field': 'ad_price_type',
                                         'type': 'string',
                                         'operator': 'in',
                                         'value': 'value.FOO,BAR'}
                                        ]},
                    'Rule6': {'condition': 'AND',
                              'rules': [{'id': 'ip_country_iso_code',
                                         'field': 'ip_country_iso_code',
                                         'type': 'string',
                                         'operator': 'equal',
                                         'value': 'field.billing_country',
                                         },
                                        {'id': 'country_id',
                                         'field': 'country_id',
                                         'type': 'string',
                                         'operator': 'not_equal',
                                         'value': 'field.ip_country_iso_code',
                                         }]}}

In [16]:
scores = {
    'Rule1': -100,
    'Rule2': -50,
    'Rule3': -25,
    'Rule4': -12,
    'Rule5': -6,
    'Rule6': -3,
}

In [17]:
c = CreateNewConfigs(conditions=system_dicts, 
                     scores=scores, 
                     app_prefix='james_testing', 
                     entity='transaction', 
                     make_active=True)

In [18]:
c.generate()

{'Rule1': {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-21T14:55:58',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule1',
  'conditions': '{"condition": "AND", "rules": [{"condition": "OR", "rules": [{"id": "payer_id.sum_approved_txn_amt_per_paypalid_1day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_1day", "type": "double", "operator": "greater_or_equal", "value": "value.60.0"}, {"id": "payer_id.sum_approved_txn_amt_per_paypalid_7day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_7day", "type": "double", "operator": "greater", "value": "value.120.0"}, {"id": "payer_id.sum_approved_txn_amt_per_paypalid_30day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_30day", "type": "double", "operator": "less_or_equal", "value": "value.500.0"}]}, {"id": "num_items", "field": "num_items", "type": "integer", "operator": "equal", "value": "value.1"}]}',
  'score': -100,
  'status': 'ACTIVE',
  'isAutoGenerated': True},
 'Rule2': {'cr